In [0]:
%sql
USE CATALOG content


In [0]:
%sql
DROP TABLE target.bronze_account_user_details

In [0]:
%sql
SELECT * FROM target.bronze_advertisements

In [0]:
%sql
SELECT * FROM target.silver_account_user_clean

In [0]:
%sql
SELECT * FROM target.silver_follow

In [0]:
%sql
SELECT * FROM target.time

In [0]:
%sql
SELECT t.year,  f.followed_account_id, f.follower_account_id FROM target.silver_follow f
INNER JOIN target.time t ON f.followed_at_time_id = t.time_id
WHERE f.status = "active"
--GROUP BY t.year, f.followed_account_id
ORDER BY 1,2

In [0]:
%sql
-- SUMA NARASTAJĄCA
SELECT DISTINCT t.year,  f.followed_account_id, COUNT(f.follower_account_id) OVER (PARTITION BY f.followed_account_id ORDER BY t.year)
FROM target.silver_follow f
INNER JOIN target.time t ON f.followed_at_time_id = t.time_id
WHERE f.status = "active" 
--GROUP BY t.year, f.followed_account_id
ORDER BY 1 ASC, 3 DESC

In [0]:
%sql
-- GOLDEN 1 -> TOP 1 z kazdego roku pod wzgledem zdobytych nowych obserwujacych
SELECT cte.year, cte.user, cte.number_of_followers FROM (
SELECT DISTINCT t.year,  f.followed_account_id user, COUNT(f.follower_account_id) number_of_followers, row_number() OVER (PARTITION BY t.year ORDER BY COUNT(f.follower_account_id) DESC) row_number
FROM target.silver_follow f
INNER JOIN target.time t ON f.followed_at_time_id = t.time_id
WHERE f.status = "active" 
--AND t.year IN (2019,2020)
GROUP BY t.year, f.followed_account_id
) cte
WHERE row_number = 1

In [0]:
%sql
-- GOLD 2 Podzial wydatkow reklamodawcow na reklamy w kwartałach Q1, Q2, Q3, Q4

SELECT CONCAT(t.year, '-', t.quarter) year_quart, adv.advertiser_name, COALESCE(SUM(ad.price_USD),0) USD_price, COALESCE(SUM(ad.Euro_price),0) EUR_price FROM target.silver_advertisers adv
LEFT JOIN target.silver_advertisements ad ON adv.advertiser_id = ad.advertiser_id
LEFT JOIN target.time t ON t.time_id = ad.created_at_time
GROUP BY t.year, t.quarter, adv.advertiser_name
ORDER BY 1 ASC, 3 DESC

In [0]:
%sql
-- GOLD 3 worzymy tabelę z wyliczonymi wskaźnikami popularności.Wpływ na biznes: Pozwala zasilać algorytm rekomendacji. Posty z wysokim engagement_rate w pierwszej godzinie od publikacji powinny być promowane wyżej w feedzie innych użytkowników.

SELECT t.year, t.month, p.post_id, p.author_id, p.visibility, p.language_code, r.reaction_type, COUNT(r.reaction_id) num_reacts, COUNT(c.comment_id) num_comments, COUNT(h.hashtag_id) num_hashtags FROM target.silver_posts p
LEFT JOIN target.silver_comments c ON p.post_id = c.post_id
LEFT JOIN target.silver_reactions r ON p.post_id = r.post_id
LEFT JOIN target.time t ON p.created_at_time = t.time_id
LEFT JOIN target.silver_post_hashtags h ON p.post_id = h.post_id
WHERE is_deleted = False
GROUP BY t.year, t.month, p.post_id, p.author_id, p.visibility, p.language_code, r.reaction_type
ORDER BY 1, 2 

In [0]:
%sql
SELECT cte.account_id, LISTAGG(cte.tag_text, ', ') hashtags FROM (
    SELECT DISTINCT a.account_id, h.tag_text, COUNT(ph.post_id), DENSE_RANK() OVER (PARTITION BY a.account_id ORDER BY COUNT(ph.post_id) DESC) row_number FROM target.silver_account_user a
    INNER JOIN target.silver_posts p ON a.account_id = p.author_id
    INNER JOIN target.silver_post_hashtags ph ON p.post_id = ph.post_id
    INNER JOIN target.silver_hashtags h ON ph.hashtag_id = h.hashtag_id
    GROUP BY a.account_id, h.tag_text
    ORDER BY 1, 3 DESC) cte
  WHERE cte.row_number IN (1,2,3)
  GROUP BY cte.account_id


In [0]:
%sql
--- WERSJA Z CTE
-- GOLD 4 -> W jednej komorce TOP 3 hashtagi dla kazdego usera. Personalizacja reklam i powiadomień push. Jeśli system wie, że Twoim ulubionym hashtagiem jest #dataengineering, może wysyłać Ci powiadomienia o nowych postach z tej kategorii.
WITH CTE AS (
  SELECT cte.account_id, LISTAGG(cte.tag_text, ', ') hashtags FROM (
    SELECT DISTINCT a.account_id, h.tag_text, COUNT(ph.post_id), DENSE_RANK() OVER (PARTITION BY a.account_id ORDER BY COUNT(ph.post_id) DESC) row_number FROM target.silver_account_user a
    INNER JOIN target.silver_posts p ON a.account_id = p.author_id
    INNER JOIN target.silver_post_hashtags ph ON p.post_id = ph.post_id
    INNER JOIN target.silver_hashtags h ON ph.hashtag_id = h.hashtag_id
    GROUP BY a.account_id, h.tag_text
    ORDER BY 1, 3 DESC) cte
  WHERE cte.row_number IN (1,2,3)
  GROUP BY cte.account_id
)


SELECT DISTINCT a.account_id, CTE.hashtags FROM target.silver_account_user a
INNER JOIN target.silver_posts p ON a.account_id = p.author_id
INNER JOIN target.silver_post_hashtags ph ON p.post_id = ph.post_id
INNER JOIN target.silver_hashtags h ON ph.hashtag_id = h.hashtag_id 
LEFT JOIN CTE ON CTE.account_id = a.account_id
--INNER JOIN target.time t ON p.created_at_time = t.time_id
--GROUP BY a.account_id
ORDER BY 1


In [0]:
%sql
--- GOLD 4 wersja BEZ CTE

SELECT DISTINCT a.account_id, (
  SELECT LISTAGG(cte.tag_text, ', ') FROM (
    SELECT DISTINCT a2.account_id, h2.tag_text, COUNT(ph2.post_id), DENSE_RANK() OVER (PARTITION BY a2.account_id ORDER BY COUNT(ph2.post_id) DESC) row_number FROM target.silver_account_user a2
    INNER JOIN target.silver_posts p2 ON a2.account_id = p2.author_id
    INNER JOIN target.silver_post_hashtags ph2 ON p2.post_id = ph2.post_id
    INNER JOIN target.silver_hashtags h2 ON ph2.hashtag_id = h2.hashtag_id
    WHERE a.account_id = a2.account_id
    GROUP BY a2.account_id, h2.tag_text
    ORDER BY 1, 3 DESC) cte
  WHERE cte.row_number IN (1,2,3)
  GROUP BY cte.account_id
)  FROM target.silver_account_user a
INNER JOIN target.silver_posts p ON a.account_id = p.author_id
INNER JOIN target.silver_post_hashtags ph ON p.post_id = ph.post_id
INNER JOIN target.silver_hashtags h ON ph.hashtag_id = h.hashtag_id 
ORDER BY 1

In [0]:
%sql
--- GOLD 5 tutaj trzeba zrobic tabele z account_user i account_details 

In [0]:
import json
from pyspark.sql import functions as F
from pyspark.sql.types import _parse_datatype_string

json_dict = {
    "username":"ArjunSinghPanam",
    "displayname":"Arjun Singh Panam",
    "userId":45091142,
    "rawDescription":"Global Citizen, Actor, Director: Sky is the roof above my head, the world is the road I travel, love is my food & mother earth is my bed. Roy in @CosmosMovie",
    "descriptionUrls":[],
    "verified":False,
    "friendsCount":311,
    "statusesCount":17534,
    "favouritesCount":4269,
    "listedCount":23,
    "mediaCount":1211,
    "location":"",
    "protected":False,
    "linkUrl":"https://www.cosmosmovieofficial.com",
    "profileImageUrl":"https://pbs.twimg.com/profile_images/1215541746492461056/3De61YoQ_normal.jpg",
    "profileBannerUrl":"https://pbs.twimg.com/profile_banners/45091142/1612601766",
    "profileUrl":"https://twitter.com/ArjunSinghPanam",

    "accountMetadata": {
      "accountAge": {
        "createdYear": 2009,
        "createdMonth": 6,
        "accountAgeCategory": "very_old_account"
      },
      "verificationStatus": {
        "isVerified": False,
        "verificationConfidence": "low"
      }
    },

    "engagementMetrics": {
      "statusesPerYearEstimate": 950,
      "mediaShare": {
        "mediaToStatusRatio": 0.069
      },
      "engagementLevel": "medium"
    },

    "profileAnalysis": {
      "hasExternalLink": True,
      "descriptionLength": 175,
      "keywordsExtracted": ["Actor", "Director", "Global Citizen", "CosmosMovie"],
      "profileCompletenessScore": 0.85
    },

    "activityPatterns": {
      "postingBehavior": {
        "isHighlyActive": True,
        "activityCategory": "content_creator"
      },
      "temporalFeatures": {
        "likelyTimeZone": "unknown",
        "accountLongevityImpact": "high"
      }
    },

    "networkFeatures": {
      "influenceIndicators": {
        "influenceScore": 0.62
      },
      "networkType": "broadcast_with_interactions"
    },

    "analyticsFlags": {
      "potentialBot": False,
      "potentialInfluencer": False,
      "dataQuality": "high"
    }
}
json_string = json.dumps(json_dict)

# 3. Pobranie schematu w formacie DDL (String) przy użyciu Sparka
# Tworzymy tymczasowy DataFrame z jednym wierszem, aby wywołać funkcję
ddl_schema = spark.range(1).select(F.schema_of_json(json_string)).collect()[0][0]

# 4. Konwersja stringa DDL na obiekt StructType
final_struct = _parse_datatype_string(ddl_schema)

# Sprawdzenie wyniku
print(final_struct)

In [0]:
%sql
SELECT * FROM content.target.bronze_account_user_details